In [3]:
import os
import re
import shutil
import pandas as pd

# === CONFIG ===
base_log_dir = r"C:\Users\ns243\Documents\Academic\AI Master\Internship\Codes\log\STOCK"
base_output_dir = r"C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\Results For Paper"

# create folders
os.makedirs(base_output_dir, exist_ok=True)
raw_dir = os.path.join(base_output_dir, "Raw Predictions")
os.makedirs(raw_dir, exist_ok=True)

# --- 1. Parse logs and extract test results ---
pattern = re.compile(
    r"average,\s*acc:\s*([\d\.\-]+),\s*mae:\s*([\d\.\-]+),\s*rmse:\s*([\d\.\-]+),\s*mape:\s*([\d\.\-]+)"
)
rows = []

for root, _, files in os.walk(base_log_dir):
    for f in files:
        if f.endswith(".log") or "log_Multitask" in f:
            log_path = os.path.join(root, f)
            with open(log_path, "r", encoding="utf-8", errors="ignore") as fp:
                text = fp.read()
            matches = pattern.findall(text)
            if matches:
                acc, mae, rmse, mape = matches[-1]
                # extract run info
                rel = os.path.relpath(log_path, base_log_dir)
                rows.append(
                    {
                        "log_file": rel,
                        "acc": float(acc),
                        "mae": float(mae),
                        "rmse": float(rmse),
                        "mape": float(mape),
                    }
                )

# save summary
summary_csv = os.path.join(base_output_dir, "stockformer_summary_metrics.csv")
if rows:
    df = pd.DataFrame(rows)
    df.to_csv(summary_csv, index=False)
    print(f"Saved summary metrics: {summary_csv}")

    # also average summary
    avg = df.mean(numeric_only=True)
    avg.to_csv(os.path.join(base_output_dir, "stockformer_summary_average.csv"))
    print("Saved average metrics.")
else:
    print("No metrics found in logs.")

# --- 2. Copy raw predictions ---
src_base = r"C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\output"
targets = [
    "classification_pred_last_step.csv",
    "classification_label_last_step.csv",
    "regression_pred_last_step.csv",
    "regression_label_last_step.csv",
]

for root, _, files in os.walk(src_base):
    for f in files:
        if f in targets:
            src = os.path.join(root, f)
            dest = os.path.join(raw_dir, f)
            try:
                shutil.copy2(src, dest)
            except Exception as e:
                print(f"Failed copying {src}: {e}")

print("All data collected into:", base_output_dir)


Saved summary metrics: C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\Results For Paper\stockformer_summary_metrics.csv
Saved average metrics.
All data collected into: C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\Results For Paper


In [4]:
import os
import json
import shutil
import pandas as pd

# === CONFIG ===
distill_dir = r"C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\distill_output_dual"
results_dir = r"C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\Results For Paper"
os.makedirs(results_dir, exist_ok=True)

# --- 1. Parse metrics JSON ---
metrics_path = os.path.join(distill_dir, "metrics_stockformer_dual.json")
if not os.path.exists(metrics_path):
    raise FileNotFoundError(f"metrics file not found: {metrics_path}")

with open(metrics_path, "r") as f:
    metrics = json.load(f)

rows = []
for task, models in metrics.items():
    for model, vals in models.items():
        rows.append({
            "task": task,
            "model": model,
            "acc": vals["ACC"],
            "mae": vals["MAE"],
            "rmse": vals["RMSE"],
            "mape": vals["MAPE"],
        })

df = pd.DataFrame(rows)
summary_csv = os.path.join(results_dir, "distillation_summary_metrics.csv")
df.to_csv(summary_csv, index=False)
print(f"Saved distillation summary metrics: {summary_csv}")

# --- 2. Copy prediction CSVs ---
targets = [
    "test_predictions_regression.csv",
    "test_predictions_classification.csv"
]

raw_dir = os.path.join(results_dir, "Distillation Predictions")
os.makedirs(raw_dir, exist_ok=True)

for f in targets:
    src = os.path.join(distill_dir, f)
    if os.path.exists(src):
        shutil.copy2(src, os.path.join(raw_dir, f))
        print(f"Copied {f}")
    else:
        print(f"Missing {f}, skipped")

print("All distillation data collected into:", results_dir)


Saved distillation summary metrics: C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\Results For Paper\distillation_summary_metrics.csv
Copied test_predictions_regression.csv
Copied test_predictions_classification.csv
All distillation data collected into: C:\Users\ns243\Documents\Academic\AI Master\Internship\Data\Results For Paper
